In [30]:
import requests, bs4, re
#1. Make a request to: `https://en.wikipedia.org/wiki/Alan_Turing` and print out the responses .text property

req = requests.get("https://en.wikipedia.org/wiki/Alan_Turing")
soup = bs4.BeautifulSoup(req.text, 'html')

#print(soup.text)

#2. Find and print the title of the response page
title = soup.find('title')
#print(title)

#3. Find and print content of the first p tag that has content.
p = soup.find('p', {"class": ""})
#print(p)

#4. Find and print all content from the TOC
table = soup.find(lambda tag: tag.name=='table')
#print(table)


#5. Create a dictionary from the TOC links like: {'first link':'#this_is_the_first_link}
dict = {}

rows = table.findChildren(['th','tr'])

for row in rows:
    cells = row.findChildren('a', href=True)
    for cell in cells:
        dict[cell.text] = cell['href']

print(dict)

#6. Create a function that can take a word and look for it in the dictionary keys and then return the content from the first link that matches. Return the next p elements until next headline (h3 element).





{'OBE': '/wiki/Officer_of_the_Order_of_the_British_Empire', 'FRS': '/wiki/Fellow_of_the_Royal_Society', '': '/wiki/File:Alan_Turing_signature.svg', 'Maida Vale': '/wiki/Maida_Vale', 'Wilmslow': '/wiki/Wilmslow', 'Cyanide poisoning': '/wiki/Cyanide_poisoning', '[note 1]': '#cite_note-1', 'Woking Crematorium': '/wiki/Woking_Crematorium', 'Sherborne School': '/wiki/Sherborne_School', 'University of Cambridge': '/wiki/University_of_Cambridge', 'BA': '/wiki/Bachelor_of_Arts', 'MA': '/wiki/Master_of_Arts', 'Princeton University': '/wiki/Princeton_University', 'Cryptanalysis of the Enigma': '/wiki/Cryptanalysis_of_the_Enigma', "Turing's proof": '/wiki/Turing%27s_proof', 'Turing machine': '/wiki/Turing_machine', 'Turing test': '/wiki/Turing_test', 'Unorganised machine': '/wiki/Unorganized_machine', 'Turing pattern': '/wiki/Turing_pattern', 'Turing reduction': '/wiki/Turing_reduction', 'The Chemical Basis of Morphogenesis': '/wiki/The_Chemical_Basis_of_Morphogenesis', 'Joan Clarke': '/wiki/Joan

In [104]:
import re, bs4
req = requests.get("https://en.wikipedia.org/wiki/Alan_Turing")
soup = bs4.BeautifulSoup(req.text, 'html')

#1. Using regex find out how many times 'Turing' is used in the article
number_of_turning = 0
pattern = 'Turing'
text = soup.select('div > p')
for lort in text:
    count = len(re.findall(pattern, lort.text))
    number_of_turning += count

print(f'The word "Turning" was used {number_of_turning} times')


#2. Regex: Find all the sentences that has a year in them (sentense defined by: starting at \n or dot or comma and ending at dot or comma.
year_pattern = r"\b(19|20)\d{2}\b"
sentence_pattern = "(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s"
text = soup.select('div > p')
for p in text:
    match = re.split(sentence_pattern, p.text)
    for sentence in match:
        if bool(re.search(year_pattern, sentence)):
            print("\nthis sentence contains a year: \n"+sentence)





The word "Turning" was used 197 times

this sentence contains a year: 
Alan Mathison Turing OBE FRS (/ˈtjʊərɪŋ/; 23 June 1912 – 7 June 1954) was an English mathematician, computer scientist, logician, cryptanalyst, philosopher, and theoretical biologist.[6] Turing was highly influential in the development of theoretical computer science, providing a formalisation of the concepts of algorithm and computation with the Turing machine, which can be considered a model of a general-purpose computer.[7][8][9] He is widely considered to be the father of theoretical computer science and artificial intelligence.[10]


this sentence contains a year: 
In 1938, he obtained his PhD from the Department of Mathematics at Princeton University.

this sentence contains a year: 
In 1948, Turing joined Max Newman's Computing Machine Laboratory, at the Victoria University of Manchester, where he helped develop the Manchester computers[13] and became interested in mathematical biology.

this sentence contain

In [138]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

#1. Use selenium to go to amazon.com and search for `Pet Shower Cap - Waterproof Reusable Bath Ear Covers`
url = 'https://www.amazon.com/'

#Chrome options
options = webdriver.ChromeOptions()
options.add_argument('--headless')

#Create a driver object using driver_path as a parameter
driver = webdriver.Chrome(options = options, service = Service(ChromeDriverManager().install()))

driver.get(url)

#will find and type keyword into search input
keyword = "Pet Shower Cap - Waterproof Reusable Bath Ear Covers"
search_box = driver.find_element(By.ID, 'twotabsearchtextbox')
search_box.send_keys(keyword)

#Will find and click the search button
search_button = driver.find_element(By.ID, 'nav-search-submit-button')
search_button.click()

# wait for the page to download
driver.implicitly_wait(5)

#2. Print how many products were found
items = wait(driver, 10).until(EC.presence_of_all_elements_located((By.XPATH, '//div[contains(@class, "s-result-item s-asin")]')))

print(f"There were {len(items)} results for {keyword}")

#3. Find the cheapest and the most expensive product from the list
#dict {link, price}
prices = {}
for item in items:
    #getting prices
    whole_price = item.find_element(By.XPATH, './/span[@class="a-price-whole"]')
    fraction_price = item.find_element(By.XPATH, './/span[@class="a-price-fraction"]')
    price = '.'.join([whole_price.text, fraction_price.text])
    #getting link
    link = item.find_element(By.XPATH, './/a[@class="a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal"]').get_attribute("href")
    prices[link] = float(price)


print(f"\nThe most expensive Product: \n{max(prices, key=prices.get)}")
print(f"\nThe cheapest Product: \n{min(prices, key=prices.get)}")

driver.quit()


There were 48 results for Pet Shower Cap - Waterproof Reusable Bath Ear Covers

The most expensive Product: 
https://www.amazon.com/Waterproof-Reusable-Elastic-Coconut-Pattern/dp/B08W565JY8/ref=sr_1_29?keywords=Pet+Shower+Cap+-+Waterproof+Reusable+Bath+Ear+Covers&qid=1668278719&sr=8-29

The cheapest Product: 
https://www.amazon.com/Waterproof-Plastic-Caps%EF%BC%8CElastic-Reusable-Bathing/dp/B09PLD3S4H/ref=sr_1_28?keywords=Pet+Shower+Cap+-+Waterproof+Reusable+Bath+Ear+Covers&qid=1668278719&sr=8-28
